In [15]:
import os, time, json,sys
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from openai import AzureOpenAI

from azure.ai.projects.models import AzureAISearchTool, ConnectionType, BingGroundingTool,ToolSet, FunctionTool, ToolSet, CodeInterpreterTool, RequiredFunctionToolCall, SubmitToolOutputsAction, ToolOutput

from opentelemetry import trace
from azure.monitor.opentelemetry import configure_azure_monitor

from dotenv import load_dotenv
import pandas as pd
load_dotenv(override=True)

from agent_team import AgentTeam
from typing import Any, Callable, Set
from user_functions import *

### Bing Search API

In [ ]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

# [START create_agent_with_azure_ai_search_tool]
conn_list = project_client.connections.list()
conn_id = ""
for conn in conn_list:
    if conn.connection_type == ConnectionType.AZURE_AI_SEARCH:
        conn_id = conn.id
        break

print(conn_id)

index_name = os.environ["AZURE_SEARCH_INDEX_NAME"].lower().replace("_", "-")

# Initialize agent AI search tool and add the search index connection id
ai_search = AzureAISearchTool(index_connection_id=conn_id, index_name=index_name)

# Create agent with AI search tool and process assistant run
with project_client:
    agent = project_client.agents.create_agent(
        model=os.environ["MODEL_DEPLOYMENT_NAME"],
        name="my-assistant",
        instructions="You are a helpful assistant",
        tools=ai_search.definitions,
        tool_resources=ai_search.resources,
        headers={"x-ms-enable-preview": "true"},
    )
    # [END create_agent_with_azure_ai_search_tool]
    print(f"Created agent, ID: {agent.id}")

    # Create thread for communication
    thread = project_client.agents.create_thread()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content="What was Microsofts revenue in 2023?",
    )
    print(f"Created message, ID: {message.id}")

    # Create and process agent run in thread with tools
    run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Delete the assistant when done
    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")

    # Fetch and log all messages
    messages = project_client.agents.list_messages(thread_id=thread.id)
    print(f"Messages: {messages}")

/subscriptions/5784b6a5-de3f-4fa4-8b8f-e5bb70ff6b25/resourceGroups/AzureAIAgents/providers/Microsoft.MachineLearningServices/workspaces/project-demo-qzzq/connections/hub-demo-qzzq-connection-AISearch


### sample_agents_run_with_toolset

In [12]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

# Enable console tracing
# or, if you have local OTLP endpoint running, change it to
# project_client.telemetry.enable(destination="http://localhost:4317")
project_client.telemetry.enable(destination=sys.stdout)

scenario = os.path.basename("simple_agent.txt")
tracer = trace.get_tracer("simple_agent_tracer")

with tracer.start_as_current_span(scenario):
    with project_client:
        agent = project_client.agents.create_agent(
            model=os.environ["MODEL_DEPLOYMENT_NAME"], name="my-assistant", instructions="You are helpful assistant"
        )
        print(f"Created agent, agent ID: {agent.id}")

        thread = project_client.agents.create_thread()
        print(f"Created thread, thread ID: {thread.id}")

        message = project_client.agents.create_message(
            thread_id=thread.id, role="user", content="Hello, tell me a joke"
        )
        print(f"Created message, message ID: {message.id}")

        run = project_client.agents.create_run(thread_id=thread.id, assistant_id=agent.id)

        # Poll the run as long as run status is queued or in progress
        while run.status in ["queued", "in_progress", "requires_action"]:
            # Wait for a second
            time.sleep(1)
            run = project_client.agents.get_run(thread_id=thread.id, run_id=run.id)

            print(f"Run status: {run.status}")

        project_client.agents.delete_agent(agent.id)
        print("Deleted agent")

        messages = project_client.agents.list_messages(thread_id=thread.id)
        print(f"messages: {messages}")

        print(messages['data'][0]['content'][0]['text']['value'])

Created agent, agent ID: asst_kgGOiplz7gicLUCxYAI294XU
Created thread, thread ID: thread_0TqND8o2CsIJqBt4pIilaDUN
Created message, message ID: msg_ibSiZwYSDlNn1ChqicpUNyvr
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.COMPLETED
Deleted agent
messages: {'object': 'list', 'data': [{'id': 'msg_7nl7aB7mfKtiUgtEvXaflPWF', 'object': 'thread.message', 'created_at': 1737691045, 'assistant_id': 'asst_kgGOiplz7gicLUCxYAI294XU', 'thread_id': 'thread_0TqND8o2CsIJqBt4pIilaDUN', 'run_id': 'run_VuyTAeUYCqYBZiSOycMiOOLY', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': "Sure! Here's a lighthearted joke for you:\n\nWhy did the scarecrow win an award?\n\nBecause he was outstanding in his field!", 'annotations': []}}], 'attachments': [], 'metadata': {}}, {'id': 'msg_ibSiZwYSDlNn1ChqicpUNyvr', 'object': 'thread.message', 'created_at': 1737691042, 'assistant_id': None, 'thread_id': 'thread_0TqND8o2CsIJqBt4pIilaDUN', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 

### sample_agents_run_with_toolset

In [2]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)
# Create agent with toolset and process assistant run
with project_client:
    # Initialize agent toolset with user functions and code interpreter
    # [START create_agent_toolset]
    functions = FunctionTool(user_functions)
    code_interpreter = CodeInterpreterTool()

    toolset = ToolSet()
    toolset.add(functions)
    toolset.add(code_interpreter)

    agent = project_client.agents.create_agent(
        model=os.environ["MODEL_DEPLOYMENT_NAME"],
        name="my-assistant",
        instructions="You are a helpful assistant",
        toolset=toolset,
    )
    # [END create_agent_toolset]
    print(f"Created agent, ID: {agent.id}")

    # Create thread for communication
    thread = project_client.agents.create_thread()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content="Hello, Find the longest word in each of these sentences: ['The quick brown fox jumps over the lazy dog', 'Python is an amazing programming language', 'Azure AI capabilities are impressive'].",
    )
    print(f"Created message, ID: {message.id}")

    # Create and process agent run in thread with tools
    # [START create_and_process_run]
    run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
    # [END create_and_process_run]
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Delete the assistant when done
    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")

    # Fetch and log all messages
    messages = project_client.agents.list_messages(thread_id=thread.id)
    print(f"Messages: {messages}")

Created agent, ID: asst_BDKifWVOklIyK1odVBCwMzXB
Created thread, ID: thread_UnMS9w8CNTt6odHv9x7gGmq4
Created message, ID: msg_YsCjL9ClC1YYhenxVal5RRvc
Run finished with status: RunStatus.COMPLETED
Deleted agent
Messages: {'object': 'list', 'data': [{'id': 'msg_gzMymZQTncEaLS8Kq1OabRZk', 'object': 'thread.message', 'created_at': 1737690383, 'assistant_id': 'asst_BDKifWVOklIyK1odVBCwMzXB', 'thread_id': 'thread_UnMS9w8CNTt6odHv9x7gGmq4', 'run_id': 'run_fvMEoVC65U3lehNi1mUSs0ql', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': 'Here are the longest words in each sentence:\n\n1. "The quick brown fox jumps over the lazy dog": **quick**\n2. "Python is an amazing programming language": **programming**\n3. "Azure AI capabilities are impressive": **capabilities**', 'annotations': []}}], 'attachments': [], 'metadata': {}}, {'id': 'msg_YsCjL9ClC1YYhenxVal5RRvc', 'object': 'thread.message', 'created_at': 1737690378, 'assistant_id': None, 'thread_id': 'thread_UnMS9w8CNTt6odHv9x7g

In [3]:
print(messages['data'][0]['content'][0]['text']['value'])

Here are the longest words in each sentence:

1. "The quick brown fox jumps over the lazy dog": **quick**
2. "Python is an amazing programming language": **programming**
3. "Azure AI capabilities are impressive": **capabilities**


### Functions with Azure Monitor

In [7]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

# Enable Azure Monitor tracing
application_insights_connection_string = project_client.telemetry.get_connection_string()
if not application_insights_connection_string:
    print("Application Insights was not enabled for this project.")
    print("Enable it via the 'Tracing' tab in your AI Foundry project page.")
    exit()
configure_azure_monitor(connection_string=application_insights_connection_string)

scenario = os.path.basename("test.txt")
tracer = trace.get_tracer("test_tracer")

@tracer.start_as_current_span("fetch_weather")  # type: ignore
def fetch_weather(location: str) -> str:
    """
    Fetches the weather information for the specified location.

    :param location (str): The location to fetch weather for.
    :return: Weather information as a JSON string.
    :rtype: str
    """
    # In a real-world scenario, you'd integrate with a weather API.
    # Here, we'll mock the response.
    mock_weather_data = {"New York": "Sunny, 25°C", "London": "Cloudy, 18°C", "Tokyo": "Rainy, 22°C"}

    # Adding attributes to the current span
    span = trace.get_current_span()
    span.set_attribute("requested_location", location)

    weather = mock_weather_data.get(location, "Weather data not available for this location.")
    weather_json = json.dumps({"weather": weather})
    return weather_json


# Statically defined user functions for fast reference
user_functions: Set[Callable[..., Any]] = {
    fetch_weather,
}

# Initialize function tool with user function
functions = FunctionTool(functions=user_functions)

with tracer.start_as_current_span(scenario):
    with project_client:
        # Create an agent and run user's request with function calls
        agent = project_client.agents.create_agent(
            model=os.environ["MODEL_DEPLOYMENT_NAME"],
            name="my-assistant",
            instructions="You are a helpful assistant",
            tools=functions.definitions,
        )
        print(f"Created agent, ID: {agent.id}")

        thread = project_client.agents.create_thread()
        print(f"Created thread, ID: {thread.id}")

        message = project_client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content="Hello, what is the weather in New York?",
        )
        print(f"Created message, ID: {message.id}")

        run = project_client.agents.create_run(thread_id=thread.id, assistant_id=agent.id)
        print(f"Created run, ID: {run.id}")

        while run.status in ["queued", "in_progress", "requires_action"]:
            time.sleep(1)
            run = project_client.agents.get_run(thread_id=thread.id, run_id=run.id)

            if run.status == "requires_action" and isinstance(run.required_action, SubmitToolOutputsAction):
                tool_calls = run.required_action.submit_tool_outputs.tool_calls
                if not tool_calls:
                    print("No tool calls provided - cancelling run")
                    project_client.agents.cancel_run(thread_id=thread.id, run_id=run.id)
                    break

                tool_outputs = []
                for tool_call in tool_calls:
                    if isinstance(tool_call, RequiredFunctionToolCall):
                        try:
                            output = functions.execute(tool_call)
                            tool_outputs.append(
                                ToolOutput(
                                    tool_call_id=tool_call.id,
                                    output=output,
                                )
                            )
                        except Exception as e:
                            print(f"Error executing tool_call {tool_call.id}: {e}")

                print(f"Tool outputs: {tool_outputs}")
                if tool_outputs:
                    project_client.agents.submit_tool_outputs_to_run(
                        thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs
                    )

            print(f"Current run status: {run.status}")

        print(f"Run completed with status: {run.status}")

        # Delete the agent when done
        project_client.agents.delete_agent(agent.id)
        print("Deleted agent")

        # Fetch and log all messages
        messages = project_client.agents.list_messages(thread_id=thread.id)
        print(f"Messages: {messages}")


Created agent, ID: asst_46M0PznVeobaH4bJVF4bNyzA
Created thread, ID: thread_htequhB9iaHFwWggSAwAqIYB
Created message, ID: msg_yQ1WPmF98uYTTyUWRsYwosnO
Created run, ID: run_Awhap06QKHcecGgnlNWaNpCu
Tool outputs: [{'tool_call_id': 'call_nbDG0NBHG6x8mRXfebkVtytc', 'output': '{"weather": "Sunny, 25\\u00b0C"}'}]
Current run status: RunStatus.REQUIRES_ACTION
Current run status: RunStatus.COMPLETED
Run completed with status: RunStatus.COMPLETED
Deleted agent
Messages: {'object': 'list', 'data': [{'id': 'msg_rjtk5hGnySQFHiDI5wbsJNBQ', 'object': 'thread.message', 'created_at': 1737690687, 'assistant_id': 'asst_46M0PznVeobaH4bJVF4bNyzA', 'thread_id': 'thread_htequhB9iaHFwWggSAwAqIYB', 'run_id': 'run_Awhap06QKHcecGgnlNWaNpCu', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': 'The weather in New York is currently sunny with a temperature of 25°C.', 'annotations': []}}], 'attachments': [], 'metadata': {}}, {'id': 'msg_yQ1WPmF98uYTTyUWRsYwosnO', 'object': 'thread.message', 'create

### Simple-Agent Team

In [9]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

with project_client:
    agent_team = AgentTeam("test_team_simple", project_client=project_client)
    agent_team.add_agent(
        model="gpt-4o",
        name="Coder",
        instructions="You are software engineer who writes great code. Your name is Coder.",
    )
    agent_team.add_agent(
        model="gpt-4o",
        name="Reviewer",
        instructions="You are software engineer who reviews code. Your name is Reviewer.",
    )
    agent_team.assemble_team()

    print("A team of agents specialized in software engineering is available for requests.")
    while True:
        user_input = input("Input (type 'quit' to exit): ")
        if user_input.lower() == "quit":
            break
        agent_team.process_request(request=user_input)

    agent_team.dismantle_team()

A team of agents specialized in software engineering is available for requests.
Created thread with ID: thread_UI5XIYhh2s9H7eh8jfo8YWFy
Starting task for agent 'TeamLeader'. Requestor: 'user'. Task description: 'Please create a task for agent in the team that is best suited to next process the following request. 
Use the create_task function available for you to create the task. The request is: 
f"Can you create a script that can read a .csv file and create a pandas dataframe out of it?"
'.
Created message with ID: msg_WQB7NExImL5dqsxdwX3M46zf for task in thread thread_UI5XIYhh2s9H7eh8jfo8YWFy
Created and processed run for agent 'TeamLeader', run ID: run_eYzpRIkcIaMf9obk0pybzzr2
Agent 'TeamLeader' completed task. Outcome: I have assigned the task to "Coder" to create a script that can read a .csv file and create a pandas DataFrame. Coder is well-suited for this task due to his expertise in coding.
Starting task for agent 'Coder'. Requestor: 'TeamLeader'. Task description: 'Please creat

### Multi-Agent Team

In [11]:
user_function_set_1: Set = {fetch_current_datetime, fetch_weather}

user_function_set_2: Set = {
    send_email_using_recipient_name,
    calculate_sum,
    toggle_flag,
    merge_dicts,
    get_user_info,
    longest_word_in_sentences,
    process_records,
}

user_function_set_3: Set = {convert_temperature}


project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

with project_client:

    functions = FunctionTool(functions=user_function_set_1)
    toolset1 = ToolSet()
    toolset1.add(functions)

    agent_team = AgentTeam("test_team_", project_client=project_client)

    agent_team.add_agent(
        model="gpt-4o",
        name="TimeWeatherAgent",
        instructions="You are a specialized agent for time and weather queries.",
        toolset=toolset1,
        can_delegate=True,
    )

    functions = FunctionTool(functions=user_function_set_2)
    toolset2 = ToolSet()
    toolset2.add(functions)

    agent_team.add_agent(
        model="gpt-4o",
        name="SendEmailAgent",
        instructions="You are a specialized agent for sending emails.",
        toolset=toolset2,
        can_delegate=False,
    )

    functions = FunctionTool(functions=user_function_set_3)
    toolset3 = ToolSet()
    toolset3.add(functions)

    agent_team.add_agent(
        model="gpt-4o",
        name="TemperatureAgent",
        instructions="You are a specialized agent for temperature conversion.",
        toolset=toolset3,
        can_delegate=False,
    )

    agent_team.assemble_team()

    user_request = (
        "Hello, Please provide me current time in '2023-%m-%d %H:%M:%S' format, and the weather in New York. "
        "Finally, convert the Celsius to Fahrenheit and send an email to Example Recipient with summary of results."
    )

    # Once process_request is called, the TeamLeader will coordinate.
    # The loop in process_request will pick up tasks from the queue, assign them, and so on.
    agent_team.process_request(request=user_request)

    agent_team.dismantle_team()

Created thread with ID: thread_jUqe0psqW2Vy3yWmOo4bL62T
Starting task for agent 'TeamLeader'. Requestor: 'user'. Task description: 'Please create a task for agent in the team that is best suited to next process the following request. 
Use the create_task function available for you to create the task. The request is: 
f"Hello, Please provide me current time in '2023-%m-%d %H:%M:%S' format, and the weather in New York. Finally, convert the Celsius to Fahrenheit and send an email to Example Recipient with summary of results."
'.
Created message with ID: msg_Vh8bs7Lns7RCMjwhXdbNzyqC for task in thread thread_jUqe0psqW2Vy3yWmOo4bL62T
Created and processed run for agent 'TeamLeader', run ID: run_dqI9kXsRQRZSen2fmNsCb1Sb
Agent 'TeamLeader' completed task. Outcome: I assigned the tasks to multiple team members as follows:

1. **TimeWeatherAgent**: Tasked with providing the current time in '2023-%m-%d %H:%M:%S' format and the weather in New York. This agent specializes in time and weather queri